In [1]:
from libs import *
from model import *
import json

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: {}".format(device))

device: cuda


# Test data processing

In [3]:
matches = pd.read_csv("perf_results.csv")
M = matches['match_id'].unique()

In [4]:
A = matches[['match_id','point_no','Alpha 1', 'Alpha 2']].loc[(matches['match_id'] == M[20])]

In [5]:
np.random.seed(0)
np.random.shuffle(M)
train_matches = M[:int(0.9*len(M))]
val_matches = M[int(0.9*len(M)):]

# Dataset

In [6]:
# Create dataset object for each point in match
class Tennis_Dataset(Dataset):
    def __init__(self, phase,  path_db, path_lb, matchs):
        super(Tennis_Dataset, self).__init__()
        self.phase = phase
        
        lb = pd.read_csv(path_lb)
        lb = lb[lb['match_id'].isin(matchs)]
        
        self.path_db = path_db
        self.result = lb
        self.output = ['Alpha 1', 'Alpha 2']
        
        self.json_data = {}
        for match_id in matchs:
            json_file_path = os.path.join(self.path_db, f"{match_id}.json")
            with open(json_file_path, "r") as file:
                self.json_data[match_id] = json.load(file)
        
    def __len__(self):
        return len(self.result)
    
    def __getitem__(self, idx):
        row = self.result.iloc[idx]
        id, point_no = row['match_id'], row['point_no']
        point_no = f'point_{point_no}'
        
        data = self.json_data[id]
        point_data = data[point_no]
        
        A = np.array(list(data[point_no].values())).T
        
        result = np.zeros((400,81))
        result[:A.shape[0],:A.shape[1]] = A
        
        labels = row[self.output].to_numpy(dtype=np.float32)
        
        return torch.from_numpy(result).float(), torch.from_numpy(labels).float()
    
    
    

In [7]:
val_dataset = Tennis_Dataset('val', 'data', "perf_results.csv", val_matches)
val_dataloader = DataLoader(val_dataset, batch_size= 3, shuffle=False)

# Validation

In [8]:
from utlis import Train, Test, static
from model import Regression_sequential_data, resnet_1d

In [9]:
criterion = nn.MSELoss()

In [10]:
def table_acc(file,net,val_loader,criterion,device):
    data=[]      
    net.load_state_dict(torch.load(path+file, map_location=device))
    size= static.get_size(path+file ,'mb')
    
    start = 1
    RMSE_loss = Test.test_model(val_loader, net, criterion, device)
    stop = 2
    time=stop - start
    data = [file.split(".")[0],RMSE_loss,size,time]
            
    # SNR,AF,IAVB,LBBB,RBBB,STD        
    col_names = ["Model", "RMSE_loss",'Size(MB)','Time(s)']
    return data,col_names



In [11]:
import os
path = os.getcwd()
path += '/result/'
os.listdir(path)

['GRU.plt', 'RNN.plt', 'Resnet.plt', 'LSTM.plt']

In [12]:
dic_model = {'RNN.plt': Regression_sequential_data.RegressionRNN(num_sensors= 81, hidden_units=16).to(device),
             'GRU.plt': Regression_sequential_data.RegressionGRU(num_sensors= 81, hidden_units=16).to(device),
             'LSTM.plt': Regression_sequential_data.RegressionLSTM(num_sensors= 81, hidden_units=16).to(device),
             'Resnet.plt': resnet_1d.ResNet18(num_classes=2, channels=81).to(device)}

In [17]:
data_report = []
for file in os.listdir(path):
    print(file)
    if file == 'Resnet.plt':
        continue    
    net = dic_model[file]
    data, col_names = table_acc(file,net,val_dataloader,criterion,device)
    data_report.append(data)



GRU.plt


Test loss: 0.013295840683935262
RNN.plt
Test loss: 0.013301118469607385
Resnet.plt
LSTM.plt
Test loss: 0.013337185754733914


In [18]:
print(tabulate(data_report, headers=col_names, tablefmt="fancy_grid", showindex="always"))

╒════╤═════════╤═════════════╤════════════╤═══════════╕
│    │ Model   │   RMSE_loss │   Size(MB) │   Time(s) │
╞════╪═════════╪═════════════╪════════════╪═══════════╡
│  0 │ GRU     │   0.0132958 │      0.026 │         1 │
├────┼─────────┼─────────────┼────────────┼───────────┤
│  1 │ RNN     │   0.0133011 │      0.01  │         1 │
├────┼─────────┼─────────────┼────────────┼───────────┤
│  2 │ LSTM    │   0.0133372 │      0.034 │         1 │
╘════╧═════════╧═════════════╧════════════╧═══════════╛


In [14]:
print(tabulate(data_report, headers=col_names, tablefmt="fancy_grid", showindex="always"))

╒════╤═════════╤═════════════╤════════════╤═══════════╕
│    │ Model   │   RMSE_loss │   Size(MB) │   Time(s) │
╞════╪═════════╪═════════════╪════════════╪═══════════╡
│  0 │ GRU     │   0.0133388 │      0.026 │         1 │
├────┼─────────┼─────────────┼────────────┼───────────┤
│  1 │ RNN     │   0.0133163 │      0.01  │         1 │
├────┼─────────┼─────────────┼────────────┼───────────┤
│  2 │ LSTM    │   0.0133372 │      0.034 │         1 │
╘════╧═════════╧═════════════╧════════════╧═══════════╛
